In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.cross_validation import KFold, train_test_split
from BeautifulSoup import BeautifulSoup
from pymongo import MongoClient
from lxml import etree

In [87]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 
def word_feats(text):
    words = text.split(' ')
    return dict([(word, True) for word in words])

#Path of data files
pospath = r'C:\Anaconda\Galvanize\Minimester2-Project\txt_sentoken\pos'
negpath = r'C:\Anaconda\Galvanize\Minimester2-Project\txt_sentoken\neg'

#Other initializations
review = []
opinion = []
n_features = 100
n_topics = 2
n_top_words = 20
pos_words = []
neg_words = []

#loop through neg files
for name in os.listdir(negpath):
    #context manager
    with open(negpath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        review.append(data.replace("'",""))
        opinion.append(False)

#loop through pos files
for name in os.listdir(pospath):
    #context manager
    with open(pospath+'\\'+name, 'r') as fo:
        #read in the data
        data = fo.read()
        review.append(data.replace("'",""))
        opinion.append(True)

negfeats = [(word_feats(review[f]), 'neg') for f in xrange(0,1000)]
posfeats = [(word_feats(review[f]), 'pos') for f in xrange(1000,2000)]

feats = np.vstack((np.array(negfeats),np.array(posfeats)))
reviews = np.array(review)
opinions = np.array(opinion)

feats_train, feats_test, reviews_train, reviews_test, opinions_train, opinions_test\
            = train_test_split(feats, reviews, opinions)


kf = KFold(len(feats_train), n_folds=5)
vectorizer = TfidfVectorizer(max_features=n_features, stop_words='english')

for train_index, test_index in kf:
    trainfeats = feats_train[train_index]
    traintestfeats = feats_train[test_index]
    
    trainreviews = reviews_train[train_index]
    traintestreviews = reviews_train[test_index]
    
    trainopinions = opinions_train[train_index]
    traintestopinions = opinions_train[test_index]
    
#     classifier = NaiveBayesClassifier.train(trainfeats)
#     print 'Train accuracy:', nltk.classify.util.accuracy(classifier, traintestfeats)
#     print 'Test accuracy:', nltk.classify.util.accuracy(classifier, feats_test)
#     classifier.show_most_informative_features()
#     print traintestreviews[0]
    train_tfidf = vectorizer.fit_transform(list(trainreviews))
    traintest_tfidf = vectorizer.fit_transform(list(traintestreviews))
#     test_tfidf = vectorizer.fit_transform(list(reviews_test))
    nmf = NMF(n_components=n_topics).fit(train_tfidf)
    z = nmf.transform(traintest_tfidf).dot(nmf.components_)
    feature_names = vectorizer.get_feature_names()
    
#     print nmf.reconstruction_err_
#     print z.dot(nmf.components_)[0]
#     print feature_names[0:9]
    
    for idx, Doc in enumerate(z):
#         print "Doc #{}".format(idx)
        if traintestopinions[idx] == True:
#             print 'A'
            pos_words.extend([feature_names[i] for i in Doc.argsort()[:-n_top_words - 1:-1]])
        elif traintestopinions[idx] == False:
#             print 'B'
            neg_words.extend([feature_names[i] for i in Doc.argsort()[:-n_top_words - 1:-1]])
        
#         print [feature_names[i] for i in Doc.argsort()[:-n_top_words - 1:-1]]
    break

print pos_words
print neg_words


[u'fun', u'new', u'long', u'life', u'great', u'takes', u'trying', u'funny', u'character', u'characters', u'little', u'watch', u'role', u'best', u'old', u'quite', u'making', u'does', u'actor', u'look', u'new', u'long', u'fun', u'life', u'great', u'old', u'best', u'trying', u'actor', u'quite', u'role', u'dont', u'making', u'takes', u'like', u'character', u'watch', u'gives', u'place', u'does', u'fun', u'new', u'long', u'life', u'great', u'trying', u'takes', u'funny', u'character', u'characters', u'old', u'best', u'role', u'watch', u'quite', u'making', u'little', u'actor', u'does', u'look', u'fun', u'new', u'long', u'life', u'great', u'trying', u'takes', u'funny', u'character', u'characters', u'old', u'best', u'role', u'watch', u'quite', u'making', u'actor', u'little', u'does', u'look', u'fun', u'new', u'long', u'life', u'great', u'trying', u'takes', u'funny', u'character', u'characters', u'best', u'role', u'watch', u'old', u'little', u'quite', u'making', u'does', u'actor', u'look', u'new'